In [1]:
from tqdm import tqdm
import os
import pandas as pd
from itertools import product
import time
import warnings

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import matplotlib.pyplot as plt

In [2]:
warnings.filterwarnings(action='ignore')

In [3]:
print(*product( map(str, range(14,22)),'bm'))

('14', 'b') ('14', 'm') ('15', 'b') ('15', 'm') ('16', 'b') ('16', 'm') ('17', 'b') ('17', 'm') ('18', 'b') ('18', 'm') ('19', 'b') ('19', 'm') ('20', 'b') ('20', 'm') ('21', 'b') ('21', 'm')


In [4]:
os.listdir('./csv/')

['2014_benign_api.csv',
 '2014_malware_api.csv',
 '2015_benign_api.csv',
 '2015_malware_api.csv',
 '2016_benign_api.csv',
 '2016_malware_api.csv',
 '2017_benign_api.csv',
 '2017_malware_api.csv',
 '2018_benign_api.csv',
 '2018_malware_api.csv',
 '2019_benign_api.csv',
 '2019_malware_api.csv',
 '2020_benign_api.csv',
 '2020_malware_api.csv',
 '2021_benign_api.csv',
 '2021_malware_api.csv']

In [5]:
df_dict= dict()
for i,v in enumerate(product( map(str, range(14,22)),'bm')):
    file=os.listdir('./csv')[i]
    key = ''.join(v)
    df_dict[key]= pd.read_csv(f'./csv/{file}', index_col=0)
    print(key, file)

14b 2014_benign_api.csv
14m 2014_malware_api.csv
15b 2015_benign_api.csv
15m 2015_malware_api.csv
16b 2016_benign_api.csv
16m 2016_malware_api.csv
17b 2017_benign_api.csv
17m 2017_malware_api.csv
18b 2018_benign_api.csv
18m 2018_malware_api.csv
19b 2019_benign_api.csv
19m 2019_malware_api.csv
20b 2020_benign_api.csv
20m 2020_malware_api.csv
21b 2021_benign_api.csv
21m 2021_malware_api.csv


In [6]:
name_dict=dict()
for key in df_dict.keys():
    if key[2] == 'b':
        name_dict[key] = '20'+key[:2]+'년 Benign'
    else:
        name_dict[key] = '20'+key[:2]+'년 Malware'

In [7]:
df_dict['14b'].iloc[1000:2000]

,Landroid/accounts/AbstractAccountAuthenticator;.getAccountRemovalAllowed:(Landroid/accounts/AccountAuthenticatorResponse;Landroid/accounts/Account;)Landroid/os/Bundle;,Landroid/accounts/AccountManager;.addAccount:(Ljava/lang/String;Ljava/lang/String;[Ljava/lang/String;Landroid/os/Bundle;Landroid/app/Activity;Landroid/accounts/AccountManagerCallback;Landroid/os/Handler;)Landroid/accounts/AccountManagerFuture;,Landroid/accounts/AccountManager;.addAccountExplicitly:(Landroid/accounts/Account;Ljava/lang/String;Landroid/os/Bundle;)Z,Landroid/accounts/AccountManager;.addOnAccountsUpdatedListener:(Landroid/accounts/OnAccountsUpdateListener;Landroid/os/Handler;Z)V,Landroid/accounts/AccountManager;.blockingGetAuthToken:(Landroid/accounts/Account;Ljava/lang/String;Z)Ljava/lang/String;,Landroid/accounts/AccountManager;.clearPassword:(Landroid/accounts/Account;)V,Landroid/accounts/AccountManager;.confirmCredentials:(Landroid/accounts/Account;Landroid/os/Bundle;Landroid/app/Activity;Landroid/accounts/AccountManagerCallback;Landroid/os/Handler;)Landroid/accounts/AccountManagerFuture;,Landroid/accounts/AccountManager;.getAccounts:()[Landroid/accounts/Account;,Landroid/accounts/AccountManager;.getAccountsByType:(Ljava/lang/String;)[Landroid/accounts/Account;,Landroid/accounts/AccountManager;.getAccountsByTypeAndFeatures:(Ljava/lang/String;[Ljava/lang/String;Landroid/accounts/AccountManagerCallback;Landroid/os/Handler;)Landroid/accounts/AccountManagerFuture;,...,Ljavax/net/ssl/HttpsURLConnection;.getOutputStream:()Ljava/io/OutputStream;,Ljavax/net/ssl/HttpsURLConnection;.setRequestMethod:(Ljava/lang/String;)V,Ljavax/net/ssl/SSLSocket;.getInputStream:()Ljava/io/InputStream;,Ljavax/net/ssl/SSLSocket;.getOutputStream:()Ljava/io/OutputStream;,Lorg/w3c/dom/Document;.getDocumentElement:()Lorg/w3c/dom/Element;,Lorg/w3c/dom/Document;.getElementsByTagName:(Ljava/lang/String;)Lorg/w3c/dom/NodeList;,Lorg/w3c/dom/Document;.getElementsByTagNameNS:(Ljava/lang/String;Ljava/lang/String;)Lorg/w3c/dom/NodeList;,Lorg/w3c/dom/Element;.getAttribute:(Ljava/lang/String;)Ljava/lang/String;,Lorg/w3c/dom/Element;.getElementsByTagName:(Ljava/lang/String;)Lorg/w3c/dom/NodeList;,Lorg/w3c/dom/Element;.getElementsByTagNameNS:(Ljava/lang/String;Ljava/lang/String;)Lorg/w3c/dom/NodeList;
apkname,,,,,,,,,,,,,,,,,,,,,
com.amazesoft.stylecollage_9E63B0985CAA1981C829AC67DF820261,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
com.infocrib.HRInterviewQuestions_9522ebf4a99c2a4bd9349a8d4a6eb8fb,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
com.radiogammastereo.radio_3445f51956e912e5e1dec87dda755940,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,3,5,0
cl.RingENergy_ec0092c2ee63de3dc8c3bbfb6b3e6401,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,4,0,0,2,3,0
com.addgames.GreedyFatBear_EN_15aba9f1a0007aebeabb98306082840b,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
com.carroll_3140e1c6f9a03d12e1bdee648c5d7b2c,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
com.mobile9.messaging_a736f91b55ff9a8a9679aa3501da2184,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,4,1,1,1,1
com.stackexchange.marvin_e37ed317c795116fe62164ab32f77855,0,0,0,0,0,0,0,0,2,0,...,1,0,0,0,0,0,0,0,0,0


# 연도 예측

In [8]:
ytrain  = [14]*8000 + [15]*8000 + [16]*8000 + [17]*8000 + [18]*8000 + [19]*8000 + [20]*8000 + [21]*8000
ytest  = [14]*2000 + [15]*2000 + [16]*2000 + [17]*2000 + [18]*2000 + [19]*2000 + [20]*2000 + [21]*2000

Xtrain, Xtest=pd.DataFrame(), pd.DataFrame()
for key in tqdm(df_dict.keys()):
    for step in range(5):
        if 0 == step:
            Xtest = pd.concat( [Xtest, df_dict[key].iloc[step*1000:(step+1)*1000]])
        else:
            Xtrain = pd.concat( [Xtrain, df_dict[key].iloc[step*1000:(step+1)*1000]])
model = RandomForestClassifier(random_state=42).fit(Xtrain, ytrain)
print('accuracy', model.score(Xtrain, ytrain), model.score(Xtest, ytest))
#     print('f1score', f1_score(ytrain, eval(f'model{i}').predict(Xtrain)), f1_score(ytest, eval(f'model{i}').predict(Xtest)))
print()
time.sleep(2)

100%|██████████| 16/16 [00:31<00:00,  1.99s/it]


accuracy 0.929703125 0.748875



In [9]:
train_result_df = pd.DataFrame()
train_result_df = pd.concat([train_result_df, pd.DataFrame([ytrain,model.predict(Xtrain)]).T])
train_result_df.columns=['label','train']

In [10]:
test_result_df = pd.DataFrame()
test_result_df = pd.concat([test_result_df, pd.DataFrame([ytest,model.predict(Xtest)]).T])
test_result_df.columns=['label','test']

In [11]:
train_result_df.shape, test_result_df.shape

((64000, 2), (16000, 2))

In [12]:
train_result_df

,label,train
0,14,14
1,14,14
2,14,14
3,14,14
4,14,14
...,...,...
63995,21,21
63996,21,21
63997,21,21
63998,21,21


In [13]:
test_result_df

,label,test
0,14,14
1,14,14
2,14,14
3,14,14
4,14,14
...,...,...
15995,21,21
15996,21,21
15997,21,21
15998,21,21


In [14]:
for i, key in enumerate(df_dict.keys()):
    print(name_dict[key])
    print(train_result_df['train'].iloc[4000*i:4000*(i+1)].value_counts()/40)
    print()

2014년 Benign
14    98.925
15     0.825
18     0.125
16     0.100
19     0.025
Name: train, dtype: float64

2014년 Malware
14    99.175
15     0.600
16     0.150
17     0.050
18     0.025
Name: train, dtype: float64

2015년 Benign
15    96.600
14     1.600
16     0.875
17     0.750
18     0.150
19     0.025
Name: train, dtype: float64

2015년 Malware
15    96.30
16     3.35
14     0.30
17     0.05
Name: train, dtype: float64

2016년 Benign
16    95.550
17     1.925
15     1.550
14     0.575
18     0.300
19     0.100
Name: train, dtype: float64

2016년 Malware
16    89.450
15     4.775
17     4.750
14     0.625
18     0.400
Name: train, dtype: float64

2017년 Benign
17    95.975
18     2.375
16     0.950
15     0.500
14     0.150
19     0.050
Name: train, dtype: float64

2017년 Malware
17    95.100
18     4.125
15     0.350
16     0.225
19     0.125
21     0.050
14     0.025
Name: train, dtype: float64

2018년 Benign
18    93.775
17     5.300
16     0.400
19     0.325
15     0.125
14     0.075
N

In [15]:
for i, key in enumerate(df_dict.keys()):
    print(name_dict[key])
    print(test_result_df['test'].iloc[1000*i:1000*(i+1)].value_counts()/10)
    print()

2014년 Benign
14    83.6
15    13.4
16     1.8
19     0.5
17     0.3
18     0.3
21     0.1
Name: test, dtype: float64

2014년 Malware
14    92.2
15     6.1
16     1.7
Name: test, dtype: float64

2015년 Benign
15    61.1
14    22.8
16    12.3
17     2.9
18     0.7
19     0.2
Name: test, dtype: float64

2015년 Malware
15    74.7
16    14.6
14    10.5
17     0.1
19     0.1
Name: test, dtype: float64

2016년 Benign
16    60.0
15    17.3
17    13.4
14     6.7
18     2.1
19     0.4
21     0.1
Name: test, dtype: float64

2016년 Malware
16    68.6
15    18.7
17     7.0
14     4.1
19     0.8
18     0.5
20     0.3
Name: test, dtype: float64

2017년 Benign
17    56.3
16    18.7
18    12.4
15     7.3
14     5.0
19     0.2
21     0.1
Name: test, dtype: float64

2017년 Malware
17    86.5
18     5.8
16     4.2
15     1.8
14     0.9
19     0.8
Name: test, dtype: float64

2018년 Benign
18    69.0
17    17.1
16     7.4
15     3.2
14     2.2
19     0.7
21     0.3
20     0.1
Name: test, dtype: float64

2018년 Malwa

# malware 탐지

In [16]:
ytrain  = [0]*4000 + [1]*4000
ytest  = [0]*1000 + [1]*1000
key_list = list(df_dict.keys())

for year_i, year in enumerate(range(14,22)):
    key1, key2 = key_list[year_i*2], key_list[year_i*2+1]
    Xtrain=pd.concat([df_dict[key1].iloc[1000:], df_dict[key2].iloc[1000:]])
    Xtest=pd.concat([df_dict[key1].iloc[:1000], df_dict[key2].iloc[:1000]])
    exec(f'bmodel_{year} = RandomForestClassifier(random_state=42).fit(Xtrain, ytrain)')
    print(f'bmodel_{year}', key1,key2)
    print('accuracy', eval(f'bmodel_{year}').score(Xtrain, ytrain), eval(f'bmodel_{year}').score(Xtest, ytest))
    print('f1score', f1_score(ytrain, eval(f'bmodel_{year}').predict(Xtrain)), f1_score(ytest, eval(f'bmodel_{year}').predict(Xtest)))
    print()

bmodel_14 14b 14m
accuracy 0.99775 0.9785
f1score 0.9977539306214125 0.9786812097174021

bmodel_15 15b 15m
accuracy 0.998375 0.977
f1score 0.9983747968496061 0.9771598808341607

bmodel_16 16b 16m
accuracy 0.997125 0.9775
f1score 0.9971296642955197 0.977667493796526

bmodel_17 17b 17m
accuracy 0.99875 0.987
f1score 0.9987506246876561 0.9869215291750503

bmodel_18 18b 18m
accuracy 0.983625 0.967
f1score 0.9837730707295925 0.967098703888335

bmodel_19 19b 19m
accuracy 0.997375 0.977
f1score 0.9973746718339792 0.9767206477732794

bmodel_20 20b 20m
accuracy 0.9865 0.977
f1score 0.9865033741564609 0.9767206477732794

bmodel_21 21b 21m
accuracy 0.996 0.98
f1score 0.9960009997500624 0.9798387096774194



# 전체 성능

In [34]:
all_df = pd.DataFrame()
for key in df_dict.keys():
    all_df = pd.concat([all_df, df_dict[key].iloc[1000:]])
    
bresult_df = pd.DataFrame()
for year_i,year in tqdm(enumerate(range(28,44))):
    for num in range(4000):
        pred_year=train_result_df.iloc[num+year_i*4000]['train']
        x=list(all_df.iloc[num+year_i*4000])
        bresult=[year//2,year_i%2, pred_year, eval(f'bmodel_{pred_year}.predict([x])')[0]]
        bresult_df = pd.concat([bresult_df ,pd.DataFrame(bresult).T])
bresult_df.columns=['year','label','pred_year','pred_B/M']

16it [08:04, 30.29s/it]


In [35]:
bresult_df

,year,label,pred_year,pred_B/M
0,14,0,14,0
0,14,0,14,0
0,14,0,14,0
0,14,0,14,0
0,14,0,14,0
...,...,...,...,...
0,21,1,21,1
0,21,1,21,1
0,21,1,21,1
0,21,1,21,1


In [36]:
accuracy_score(bresult_df["year"],bresult_df["pred_year"])

0.929703125

In [37]:
# f1_score(bresult_df["year"],bresult_df["pred_year"])

In [38]:
accuracy_score(bresult_df["label"],bresult_df["pred_B/M"])

0.99259375

In [39]:
f1_score(bresult_df["label"],bresult_df["pred_B/M"])

0.9926200411035685

In [25]:
all_df = pd.DataFrame()
for key in df_dict.keys():
    all_df = pd.concat([all_df, df_dict[key].iloc[:1000]])

In [26]:
all_df.shape

(16000, 1848)

In [27]:
bresult_df = pd.DataFrame()
for year_i,year in tqdm(enumerate(range(28,44))):
    for num in range(1000):
        pred_year=test_result_df.iloc[num+year_i*1000]['test']
        x=list(all_df.iloc[num+year_i*1000])
        bresult=[year//2,year_i%2, pred_year, eval(f'bmodel_{pred_year}.predict([x])')[0]]
        bresult_df = pd.concat([bresult_df ,pd.DataFrame(bresult).T])
bresult_df.columns=['year','label','pred_year','pred_B/M']

16it [01:58,  7.42s/it]


In [28]:
bresult_df

,year,label,pred_year,pred_B/M
0,14,0,14,0
0,14,0,14,0
0,14,0,14,0
0,14,0,14,0
0,14,0,14,0
...,...,...,...,...
0,21,1,21,1
0,21,1,21,1
0,21,1,21,1
0,21,1,21,1


In [29]:
accuracy_score(bresult_df["year"],bresult_df["pred_year"])

0.748875

In [30]:
# f1_score(bresult_df["year"],bresult_df["pred_year"])

In [31]:
accuracy_score(bresult_df["label"],bresult_df["pred_B/M"])

0.9706875

In [32]:
f1_score(bresult_df["label"],bresult_df["pred_B/M"])

0.970806100217865